<a href="https://colab.research.google.com/github/tajibo/HomeworkDTS/blob/main/03_paraphrase_russian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парафраз русских предложений с Hugging Face
Этот блокнот использует модель `cointegrated/rut5-base-paraphraser` для перефразирования русских текстов.
Также используется датасет `russian-superglue/ru_paraphraser` для проверки качества модели.

In [ ]:
# Установка библиотек
# !pip install transformers datasets torch
!pip install transformers datasets

## 1. Загрузка датасета

In [ ]:
from datasets import load_dataset

# Загружаем датасет
dataset = load_dataset("merionum/ru_paraphraser")

# Проверяем структуру
print(dataset)

In [ ]:
dataset["train"][10]

In [ ]:
dataset["train"][0]

## 2. Фильтрация данных (только правильные парафразы)

In [ ]:
# Фильтруем только примеры, где label = 1 (парафраз)
train_data = dataset["train"].filter(lambda x: x["class"] == '1')
test_data = dataset["test"].filter(lambda x: x["class"] == '1')

# Выводим пример
print(f"Количество примеров: {len(train_data)}")
print(train_data.shape)

## 3. Использование `pipeline` для парафраза

In [ ]:
from transformers import pipeline

# Загружаем pipeline для парафраза
paraphraser = pipeline("text2text-generation", model="cointegrated/rut5-base-paraphraser")

# Пример работы модели
text = "Я собираюсь поехать в Москву завтра."
result = paraphraser(text, max_length=50, num_return_sequences=1)

print("Исходное:", text)
print("Перефразированное:", result[0]["generated_text"])

## 4. Проверка модели на тестовом датасете

In [ ]:
correct = 0
total = 5  # Берем 5 примеров для теста

for i in range(total):
    original = test_data[i]["text_1"]
    expected = test_data[i]["text_2"]

    # Генерируем парафраз
    generated = paraphraser(original, max_length=50, num_return_sequences=1)[0]["generated_text"]

    print(f" Исходное: {original}")
    print(f"Ожидаемое: {expected}")
    print(f"Сгенерированное: {generated}")

## Вывод
- Использовали датасет `ru_paraphraser` с Hugging Face.
- Фильтровали правильные примеры (class="1").
- Использовали `pipeline` для парафраза.
- Проверили качество модели на тестовых примерах.

Теперь можно **использовать код для генерации русских парафраз**!


# ЗАДАНИЕ
Допишите оценку модели по метрике ROUGE

In [ ]:
import evaluate

# Загружаем метрику ROUGE
rouge = evaluate.load("rouge")

# Количество тестовых примеров, по которым проводим оценку
total = 20  # можно увеличить для более надежной оценки

predictions = []
references = []

for i in range(total):
    original = test_data[i]["text_1"]
    expected = test_data[i]["text_2"]

    # Генерируем парафразу
    generated = paraphraser(original, max_length=50, num_return_sequences=1)[0]["generated_text"]

    predictions.append(generated)
    references.append(expected)

    print(f"Исходное: {original}")
    print(f"Ожидаемое: {expected}")
    print(f"Сгенерированное: {generated}\n")

# Вычисляем метрики ROUGE
results = rouge.compute(predictions=predictions, references=references)
print("Результаты ROUGE:")
for key, value in results.items():
    print(f"  {key}: {value:.4f}")
